In [ ]:
%cd '/content/drive/MyDrive/TensorFlow'

/content/drive/MyDrive/TensorFlow


In [ ]:
!git clone https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


In [ ]:
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 34 not upgraded.
Need to get 1,614 kB of archives.
After this operation, 8,907 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4

In [ ]:
#cd into 'TensorFlow/models/research'
%cd '/content/drive/MyDrive/TensorFlow/models/research'
!protoc object_detection/protos/*.proto --python_out=.

/content/drive/MyDrive/TensorFlow/models/research


In [ ]:
import os
import sys 

os.environ['PYTHONPATH']+=":/content/drive/MyDrive/TensorFlow/models"
sys.path.append("/content/drive/MyDrive/TensorFlow/models/research")


In [ ]:
!python setup.py build
!python setup.py install

running build
running build_py
copying object_detection/protos/anchor_generator_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/bipartite_matcher_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/argmax_matcher_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/box_predictor_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/calibration_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/center_net_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/faster_rcnn_box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/eval_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/flexible_grid_anchor_generator_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/faster_rcnn_pb2.py -> 

In [ ]:
#cd into 'TensorFlow/models/research/object_detection/builders/'
%cd '/content/drive/MyDrive/TensorFlow/models/research/object_detection/builders'
!python model_builder_tf2_test.py 

/content/drive/MyDrive/TensorFlow/models/research/object_detection/builders
2021-04-26 17:06:00.777806: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Running tests under Python 3.7.10: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2021-04-26 17:06:16.011953: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-26 17:06:16.013103: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-26 17:06:16.069813: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-04-26 17:06:16.070497: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P4 computeCapa

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

Done


In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config_path = '/content/drive/MyDrive/TensorFlow/new_model/pipeline.config'

In [ ]:
#loading the config file and merging it with given model's config file

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

with tf.io.gfile.GFile(config_path,'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
#fixing number of classes for detection
pipeline_config.model.ssd.num_classes = 3

#fixing the batchsize for training
pipeline_config.train_config.batch_size= 5

#transfer learning - using the checkpoints of pretrained model
pipeline_config.train_config.fine_tune_checkpoint = 'Checkpoints/checkpoint/ckpt-0' 
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection' 

#training record setting
pipeline_config.train_input_reader.label_map_path =  'annotations/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] =  ['annotations/train.record']

#testing record setting
pipeline_config.eval_input_reader[0].label_map_path = 'annotations/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = ['annotations/test.record']


In [ ]:
pipeline_config

model {
  ssd {
    num_classes: 3
    image_resizer {
      fixed_shape_resizer {
        height: 1024
        width: 1024
      }
    }
    feature_extractor {
      type: "ssd_resnet50_v1_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 0.00039999998989515007
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      

In [ ]:
#write the updated config file to directory 
#convert protobuf message to string to be written in our config file
config_text = text_format.MessageToString(pipeline_config)

with tf.io.gfile.GFile(config_path,'wb') as f:
    f.write(config_text)

In [ ]:
%cd '/content/drive/MyDrive/TensorFlow'

/content/drive/MyDrive/TensorFlow


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU') 
for gpu in gpus: 
  tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
'python /content/drive/MyDrive/TensorFlow/model_main_tf2.py --model_dir=/content/drive/MyDrive/TensorFlow/new_model --pipeline_config_path=/content/drive/MyDrive/TensorFlow/new_model/pipeline.config --num_train_steps=2000'

'python /content/drive/MyDrive/TensorFlow/model_main_tf2.py --model_dir=/content/drive/MyDrive/TensorFlow/new_model --pipeline_config_path=/content/drive/MyDrive/TensorFlow/new_model/pipeline.config --num_train_steps=2000'

In [ ]:
!python model_main_tf2.py --model_dir=new_model/new_model_1 --pipeline_config_path=new_model/pipeline.config --checkpoint_dir=new_model --num_train_steps=2000 

Traceback (most recent call last):
  File "model_main_tf2.py", line 113, in <module>
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 303, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "model_main_tf2.py", line 88, in main
    wait_interval=300, timeout=FLAGS.eval_timeout)
  File "/usr/local/lib/python3.7/dist-packages/object_detection-0.1-py3.7.egg/object_detection/model_lib_v2.py", line 1110, in eval_continuously
    checkpoint_dir, timeout=timeout, min_interval_secs=wait_interval):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/checkpoint_utils.py", line 198, in checkpoints_iterator
    checkpoint_dir, checkpoint_path, timeout=timeout)
  File "/usr/local/lib/python3.7/d